### testing the XGBoost Diabetes example to transpile
##### used conda env giza from ll laptop

## Create and Train an XGBoost Model
### We'll start by creating a simple XGBoost model using Scikit-Learn and train it on diabetes dataset

In [2]:
import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

data = load_diabetes()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Increase the number of trees and maximum depth
n_estimators = 2  # Increase the number of trees
max_depth = 6  # Increase the maximum depth of each tree

xgb_reg = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth)
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [3]:
X_train

array([[ 0.07076875,  0.05068012,  0.01211685, ...,  0.03430886,
         0.02736405, -0.0010777 ],
       [-0.00914709,  0.05068012, -0.01806189, ...,  0.07120998,
         0.00027248,  0.01963284],
       [ 0.00538306, -0.04464164,  0.04984027, ..., -0.00259226,
         0.01703607, -0.01350402],
       ...,
       [ 0.03081083, -0.04464164, -0.02021751, ..., -0.03949338,
        -0.01090325, -0.0010777 ],
       [-0.01277963, -0.04464164, -0.02345095, ..., -0.00259226,
        -0.03845972, -0.03835666],
       [-0.09269548, -0.04464164,  0.02828403, ..., -0.03949338,
        -0.00514219, -0.0010777 ]])

## Save the model
### Save the model in Json format

In [2]:
from giza.zkcook import serialize_model
serialize_model(xgb_reg, "xgb_diabetes.json")

## Transpile your model to Orion Cairo
### We will use Giza-CLI to transpile our saved model to Orion Cairo.


$ giza transpile xgb_diabetes.json --output-path xgb_diabetes
[giza][2024-05-29 23:43:03.612] No model id provided, checking if model exists ✅
[giza][2024-05-29 23:43:03.621] Model name is: xgb_diabetes
[giza][2024-05-29 23:43:04.421] Model Created with id -> 665! ✅
[giza][2024-05-29 23:43:05.200] Version Created with id -> 1! ✅
[giza][2024-05-29 23:43:05.208] Sending model for transpilation ✅
[giza][2024-05-29 23:43:16.618] Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
[giza][2024-05-29 23:43:17.750] Downloading model ✅
[giza][2024-05-29 23:43:17.758] model saved at: xgb_diabetes
(giza3)

## Deploy an inference endpoint
### Now that our model is transpiled to Cairo we can deploy an endpoint to run verifiable inferences. We will use Giza CLI again to run and deploy an endpoint. Ensure to replace model-id and version-id with your ids provided during transpilation.


giza endpoints deploy --model-id 665 --version-id 1

$ giza endpoints deploy --model-id 665 --version-id 1
▰▱▱▱▱▱▱ Creating endpoint!
[giza][2024-05-29 23:53:51.319] Endpoint is successful ✅
[giza][2024-05-29 23:53:51.335] Endpoint created with id -> 234 ✅
[giza][2024-05-29 23:53:51.346] Endpoint created with endpoint URL: https://endpoint-giza1-665-1-7ee56cfd-7i3yxzspbq-ew.a.run.app 🎉
(giza3)

## Run a verifiable inference
##### To streamline verifiable inference, you might consider using the endpoint URL obtained after transpilation. However, this approach requires manual serialization of the input for the Cairo program and handling the deserialization process. To make this process more user-friendly and keep you within a Python environment, we've introduced a Python SDK designed to facilitate the creation of ML workflows and execution of verifiable predictions. When you initiate a prediction, our system automatically retrieves the endpoint URL you deployed earlier, converts your input into Cairo-compatible format, executes the prediction, and then converts the output back into a numpy object. 

In [1]:
import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

from giza.agents.model import GizaModel


MODEL_ID = 665  # Update with your model ID
VERSION_ID = 1  # Update with your version ID

def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={"input": input}, verifiable=True, model_category="XGB"
    )

    return result, proof_id


def execution():
    # The input data type should match the model's expected input
    input = X_test[1, :]

    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(f"Predicted value for input {input.flatten()[0]} is {result}")

    return result, proof_id


if __name__ == "__main__":
    data = load_diabetes()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    _, proof_id = execution()
    print(f"Proof ID: {proof_id}")

C:\Users\chuck.raghavan\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Predicted value for input 0.09256398319871433 is 175.58781
Proof ID: fa2ebd40599b466b80b705b2f62dcc02


## Download the proof
#### Initiating a verifiable inference sets off a proving job on our server, sparing you the complexities of installing and configuring the prover yourself. Upon completion, you can download your proof.

First, let's check the status of the proving job to ensure that it has been completed.

$ giza endpoints get-proof --endpoint-id 234 --proof-id "10c164e6c2364ab6b5491702127979a6"
[giza][2024-05-30 00:40:39.691] Getting proof from endpoint 234 ✅
{
  "id": 967,
  "job_id": 1121,
  "metrics": {
    "proving_time": 17.249508
  },
  "created_date": "2024-05-30T07:33:12.532659"
}

Once the proof is ready, you can download it.

$ giza endpoints download-proof --endpoint-id 234 --proof-id "10c164e6c2364ab6b5491702127979a6" --output-path zk_xgboost.proof
[giza][2024-05-30 00:51:52.048] Getting proof from endpoint 234 ✅
[giza][2024-05-30 00:51:53.800] Proof downloaded to zk_xgboost.proof ✅
(giza3)

## Verify the proof
#### Finally, you can verify the proof.

$ giza verify --proof-id 967
[giza][2024-05-30 00:56:05.847] Verifying proof...
[giza][2024-05-30 00:56:07.140] Verification result: True
[giza][2024-05-30 00:56:07.145] Verification time: 0.454667226
(giza3)